<h1> pyCRTM Jupyter Notebook </h1>

test_atms.ipynb

<h3>Description:</h3>
Jupyter notebook version of the <b>test_atms.ipynb</b> Python test script for pyCRTM. 

<h3>Requirements:</h3>
Python 3

<h3>Record of Revisions:</h3>
    
Date: 2020-02-26 Author: P. Stegmann Description: Original code

<h2> Setup </h2>

Import Python utility modules:

In [ ]:
import configparser
import os
import h5py
import sys 
import numpy as np
from matplotlib import pyplot as plt

Add current directory to Python <i>sys<i/> path:

In [ ]:
thisDir = os.path.dirname(os.path.abspath('path'))
print(thisDir)
parentDir = os.path.dirname(thisDir)
print(parentDir)
sys.path.append(parentDir)

Load the pyCRTM module:

In [ ]:
from pyCRTM import pyCRTM, profilesCreate

<h2> Creating the Test Cases: </h2>

In [ ]:
thisDir = os.path.dirname(os.path.abspath('path'))
print(thisDir)
cases = os.listdir( os.path.join(thisDir,'data') )
print(cases)
cases.sort()

Create 4 arrays to store atmospheric profile data and precomputed brightness temperatures:

In [ ]:
profiles = profilesCreate( 4, 92)
storedTb = []
storedEmis = []

Populate the arrays with data for the 4 cases:

In [ ]:
for i,c in enumerate(cases):
        h5 = h5py.File(os.path.join(thisDir,'data',c) , 'r')
        profiles.Angles[i,0] = h5['zenithAngle'][()]
        profiles.Angles[i,1] = 999.9 
        profiles.Angles[i,2] = 100.0  # 100 degrees zenith below horizon.
        profiles.Angles[i,3] = 0.0 # zero solar azimuth 
        profiles.Angles[i,4] = h5['scanAngle'][()]
        profiles.DateTimes[i,0] = 2001
        profiles.DateTimes[i,1] = 1
        profiles.DateTimes[i,2] = 1
        profiles.Pi[i,:] = np.asarray(h5['pressureLevels'] )  # [hPa]
        profiles.P[i,:] = np.asarray(h5['pressureLayers'][()]) # [hPa]
        profiles.T[i,:] = np.asarray(h5['temperatureLayers']) # [K]
        profiles.Q[i,:] = np.asarray(h5['humidityLayers'])
        profiles.O3[i,:] = np.asarray(h5['ozoneConcLayers'])
        profiles.clouds[i,:,0,0] = np.asarray(h5['cloudConcentration'])
        profiles.clouds[i,:,0,1] = np.asarray(h5['cloudEffectiveRadius'])
        profiles.aerosols[i,:,0,0] = np.asarray(h5['aerosolConcentration'])
        profiles.aerosols[i,:,0,1] = np.asarray(h5['aerosolEffectiveRadius'])
        profiles.aerosolType[i] = h5['aerosolType'][()]
        profiles.cloudType[i] = h5['cloudType'][()]
        profiles.cloudFraction[i,:] = h5['cloudFraction'][()]
        profiles.climatology[i] = h5['climatology'][()]
        profiles.surfaceFractions[i,:] = h5['surfaceFractions']
        profiles.surfaceTemperatures[i,:] = h5['surfaceTemperatures']
        profiles.S2m[i,1] = 33.0 # just use salinity out of S2m for the moment.
        profiles.windSpeed10m[i] = 5.0
        profiles.LAI[i] = h5['LAI'][()]
        profiles.windDirection10m[i] = h5['windDirection10m'][()]
        # land, soil, veg, water, snow, ice
        profiles.surfaceTypes[i,0] = h5['landType'][()]
        profiles.surfaceTypes[i,1] = h5['soilType'][()]
        profiles.surfaceTypes[i,2] = h5['vegType'][()]
        profiles.surfaceTypes[i,3] = h5['waterType'][()]
        profiles.surfaceTypes[i,4] = h5['snowType'][()]
        profiles.surfaceTypes[i,5] = h5['iceType'][()]
        storedTb.append(np.asarray(h5['Tb_atms'][0:22]))
        storedEmis.append(np.asarray(h5['emissivity_atms'][0:22]))
        h5.close()

<h3> Inspect input data </h3>

Plot the temperature:

In [ ]:
for i,c in enumerate(cases):
    plt.semilogy(profiles.T[i,:],profiles.P[i,:],label='case %d'%(i+1))
plt.gca().invert_yaxis()
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [hPa]')
plt.legend()

Plot the water vapor concentration:

In [ ]:
for i,c in enumerate(cases):
    plt.loglog(profiles.Q[i,:],profiles.P[i,:],label='case %d'%(i+1))
plt.gca().invert_yaxis()
plt.xlabel(r'Absolute Humidity $[g/m^3]$')
plt.ylabel('Pressure [hPa]')
plt.legend()

<h2> CRTM Initialization </h2>

Read the pyCRTM configuration file:

In [ ]:
pathInfo = configparser.ConfigParser()
pathInfo.read( os.path.join(parentDir,'crtm.cfg') ) 

Read the path info of the CRTM binary coefficient files:

In [ ]:
coefficientPath = pathInfo['CRTM']['coeffs_dir']

Set the satellite instrument ID as defined in the CRTM documentation:

In [ ]:
sensor_id = 'atms_npp'

Instantiate a pyCRTM object:

In [ ]:
crtmOb = pyCRTM()

Set the data members of the pyCRTM object:

In [ ]:
crtmOb.profiles = profiles
crtmOb.coefficientPath = pathInfo['CRTM']['coeffs_dir'] # Location of the CRTM binary coefficients
crtmOb.sensor_id = sensor_id
crtmOb.nThreads = 4 

Load CRTM SpcCoeff.bin Spectral coefficient files for the selected instrument:

In [ ]:
crtmOb.loadInst()

<h2> Run the CRTM in Forward and Jacobian Mode</h2>

Run the Forward solver of the CRTM for the 4 test cases:

In [ ]:
crtmOb.runDirect()

Extract instrument brightness temperatures from solution object:

In [ ]:
forwardTb = crtmOb.Bt

Extract surface emissivity from solution object:

In [ ]:
forwardEmissivity = crtmOb.surfEmisRefl[0,:]

Reset the surface emissivity for the K-Matrix run:

In [ ]:
crtmOb.surfEmisRefl = []

Run the K-Matrix solver:

In [ ]:
crtmOb.runK()

Extract the brightness temperature solution and the surface emissivity Jacobian:

In [ ]:
kTb = crtmOb.Bt
kEmissivity = crtmOb.surfEmisRefl[0,:]

Compare the extracted solution data with the precomputed Fortran solution from the CRTM and plot the difference if substantial:

In [ ]:
if ( all( np.abs( forwardTb.flatten() - np.asarray(storedTb).flatten() ) <= 1e-5)  and all( np.abs( kTb.flatten() - np.asarray(storedTb).flatten() ) <= 1e-5) ):
    print("Yay! all values are close enough to what CRTM test program produced!")
else: 
    print("Boo! something failed. Look at atms plots")
    wavenumbers = np.zeros([4,22])
    wavenumbers[0:4,:] = np.linspace(1,23,22)
        
    plt.figure()
    plt.plot(wavenumbers.T,forwardTb.T-np.asarray(storedTb).T ) 
    plt.legend(['1','2','3','4'])
    plt.savefig(os.path.join(thisDir,'atms'+'_spectrum_forward.png'))
    plt.figure()
    plt.plot(wavenumbers.T,forwardEmissivity.T-np.asarray(storedEmis).T)
    plt.savefig(os.path.join(thisDir,'atms'+'_emissivity_forward.png')) 
    
    plt.figure()
    plt.plot(wavenumbers.T,kTb.T-np.asarray(storedTb).T)
    plt.savefig(os.path.join(thisDir,'atms'+'_spectrum_k.png'))
    plt.figure()
    plt.plot(wavenumbers.T,kEmissivity.T-np.asarray(storedEmis).T)
    plt.savefig(os.path.join(thisDir,'atms'+'_emissivity_k.png')) 
    sys.exit("Boo! didn't pass tolerance with CRTM test program.")